# Imports

In [ ]:
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler
import os
from tqdm import tqdm
from time import sleep
import cv2

# DataLoader

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

    
data_dir = '../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'
def load_split_train_test(datadir, batch_size, valid_size = .2):
         #No need to resize images since they are all 200 x 200 x 3
    train_transforms = transforms.Compose([transforms.Resize((224, 224)),transforms.ColorJitter(0.9, 0.9, 0.9, 0.1), transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    test_transforms = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    train_data = datasets.ImageFolder(datadir, transform=train_transforms)
    test_data = datasets.ImageFolder(datadir, transform=test_transforms)

    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)

    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)

    trainloader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    testloader = torch.utils.data.DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

    return trainloader, testloader

batch_size = 32
trainloader, testloader = load_split_train_test(data_dir, batch_size, .18)
print("Train Size:", len(trainloader) * batch_size, ", No of bacthes:", len(trainloader))
print("Test Size:", len(testloader) * batch_size, ", No of bacthes:", len(testloader))
print("Classes:", trainloader.dataset.classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:",device)

In [ ]:
class GaussianBlur(torch.nn.Module):
    def __init__(self, kernel_size, std_dev):
        self.kernel_size = kernel_size
        self.std_dev = std_dev

    def forward(self, img):
        image = np.array(img)
        image_blur = cv2.GaussianBlur(image, self.kernel_size, self.std_dev)
        return Image.fromarray(image_blur)

# Model Building

In [ ]:
backbone = 'mobilenet_v2'
addLayers = False
if backbone == 'resnet50':
    model = models.resnet50(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    if addLayers:
        model.fc = nn.Sequential(nn.Linear(2048, 1024),
          nn.ReLU(),
          nn.Dropout(0.2),
          nn.Linear(1024, len(trainloader.dataset.classes)),
          nn.LogSoftmax(dim=1)
          )
    else:
        model.fc = nn.Linear(2048, len(trainloader.dataset.classes))
elif backbone == 'mobilenet_v2':
    model = models.mobilenet_v2(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    if addLayers:
        model.classifier = nn.Sequential(nn.Linear(1280, 1024),
          nn.ReLU(),
          nn.Dropout(0.2),
          nn.Linear(1024, len(trainloader.dataset.classes)),
          nn.LogSoftmax(dim=1)
          )
    else:
        model.classifier = nn.Linear(1280, len(trainloader.dataset.classes))
print(model)

# Loss and Optimizer

In [ ]:
#criterion = nn.NLLLoss()
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)
model.to(device)

# Model Training

In [ ]:
epochs = 6
running_loss = 0
train_losses, test_losses = [], []
min_val_loss = None
name = ''

In [ ]:
for epoch in range(epochs):
    with tqdm(trainloader, unit="batch") as tepoch:
        accuracy = 0
        for i, data in enumerate(tepoch):
            inputs, labels = data
            tepoch.set_description(f"Training Epoch {epoch + 1}")
            inputs, labels = inputs.to(device), labels.to(device)
            size = labels.shape[0]
            optimizer.zero_grad()
            logps = model.forward(inputs)
            # print(torch.argmax(logps, dim=1).shape, labels.shape)
            loss = criterion(logps, labels)
            train_acc = torch.sum(torch.argmax(logps, dim=1) == labels).item() / size
            accuracy += train_acc
            loss.backward()

            optimizer.step()
            running_loss += loss.item()

            tepoch.set_postfix(loss=loss.item(), accuracy=100. * train_acc)
            sleep(0.005)
            if i == len(trainloader)-1:
                accuracy = accuracy / len(trainloader)
                tepoch.set_postfix(loss=running_loss/len(trainloader), accuracy=100. * accuracy)
    test_loss = 0
    accuracy = 0
    model.eval()
    with torch.no_grad():
        with tqdm(testloader, unit="batch") as tepoch:
            for i, data in enumerate(tepoch):
                (inputs, labels) = data
                tepoch.set_description(f"Testing Epoch {epoch + 1}")
                inputs, labels = inputs.to(device), labels.to(device)
                size = labels.shape[0]
                logps = model.forward(inputs)
                batch_loss = criterion(logps, labels)
                test_loss += batch_loss.item()

                test_acc = torch.sum(torch.argmax(logps, dim=1) == labels).item() / size
                accuracy += test_acc
                tepoch.set_postfix(loss=batch_loss.item(), accuracy=100. * test_acc)
                # tepoch.set_postfix(loss=batch_loss.item())
                sleep(0.005)
                if i == len(testloader)-1:
                        accuracy = accuracy / len(testloader)
                        tepoch.set_postfix(loss=test_loss/len(testloader), accuracy=100. * accuracy)

    val_loss = test_loss/len(testloader)
    if min_val_loss is None:
        min_val_loss = val_loss
        name = 'sl_recognition_{}_{}_{}.pth'.format(str(epoch + 1), str(round(val_loss, 3)), str(round(accuracy, 3)))
        torch.save(model, name)
    elif min_val_loss > val_loss:
        min_val_loss = val_loss
        name = 'sl_recognition_{}_{}_{}.pth'.format(str(epoch + 1), str(round(val_loss, 3)), str(round(accuracy, 3)))
        torch.save(model, name)

    running_loss = 0
    model.train()
torch.save(model, 'final_sl.pth')

# Generate Classes.txt

In [ ]:
with open('classes.txt', 'w') as f:
    for clas in trainloader.dataset.classes:
        f.write(clas+'\n')

# Convert to onnx

In [ ]:
name =  '../input/modelm/sl_recognition_20_0.185_0.94.pth'

onnx_model_path = "sl.onnx"
model = torch.load(name)
model.to("cpu")
model.eval()
dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(model, dummy_input, onnx_model_path, verbose=True)

# Run Inference with opencv

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def decrease_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] -= value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [ ]:
plt.figure(figsize=(20, 20))
imageDir = '../input/ankutest2'
net =  cv2.dnn.readNetFromONNX(onnx_model_path) 
with open('classes.txt', 'r') as f:
    classes = f.read().split('\n')
for i, image_name in enumerate(os.listdir(imageDir)):
    image = cv2.imread(os.path.join(imageDir, image_name))
    image = image[5:-5, 5:-5]
    image = decrease_brightness(image, value=25)
    blob = cv2.dnn.blobFromImage(image, 1.0 / 255, (224, 224),(0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    preds = net.forward()
    biggest_pred_index = np.array(preds)[0].argmax()
    ax = plt.subplot(6, 5, i + 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("predicted: {}, True: {}".format(classes[biggest_pred_index], image_name.split('_test.jpg')[0]))
    plt.axis("off")

In [ ]:
plt.figure(figsize=(20, 20))
imageDir = '../input/asl-alphabet/asl_alphabet_test/asl_alphabet_test'
net =  cv2.dnn.readNetFromONNX(onnx_model_path) 
with open('classes.txt', 'r') as f:
    classes = f.read().split('\n')
for i, image_name in enumerate(os.listdir(imageDir)):
    image = cv2.imread(os.path.join(imageDir, image_name))
#     image = image[2:2+275, 2:2+275]
#     image = image[5:-5, 5:-5]
    blob = cv2.dnn.blobFromImage(image, 1.0 / 255, (224, 224),(0, 0, 0), swapRB=False, crop=False)
    net.setInput(blob)
    preds = net.forward()
    biggest_pred_index = np.array(preds)[0].argmax()
    ax = plt.subplot(6, 5, i + 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("predicted: {}, True: {}".format(classes[biggest_pred_index], image_name.split('_test.jpg')[0]))
    plt.axis("off")

In [ ]:
plt.figure(figsize=(20, 20))
imageDir = '../input/ankutest2'
net =  cv2.dnn.readNetFromONNX(onnx_model_path) 
with open('classes.txt', 'r') as f:
    classes = f.read().split('\n')
for i, image_name in enumerate(os.listdir(imageDir)):
    image = cv2.imread(os.path.join(imageDir, image_name))
#     image = image[2:2+275, 2:2+275]
    image = image[5:-5, 5:-5]
    blob = cv2.dnn.blobFromImage(image, 1.0 / 255, (224, 224),(0, 0, 0), swapRB=False, crop=False)
    net.setInput(blob)
    preds = net.forward()
    biggest_pred_index = np.array(preds)[0].argmax()
    ax = plt.subplot(6, 5, i + 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("predicted: {}, True: {}".format(classes[biggest_pred_index], image_name.split('_test.jpg')[0]))
    plt.axis("off")